<a href="https://colab.research.google.com/github/Assylbek15/huggingface-nlp-course/blob/main/chapter01_introduction/transformers_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers, what can they do?

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:

# The `pipeline("sentiment-analysis")` line automatically:
# - Loads a pretrained model (usually `distilbert-base-uncased-finetuned-sst-2-english`)
# - Sets up the tokenizer and model for inference
# - Preprocesses the input, runs it through the model, and postprocesses the result

# input -> any sentence ,
# output -> (positive/negative) & confidence score
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("No more buses, no more waiting — just me and my Lamborghini in the fast lane of life.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.5876696705818176}]

In [ ]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
classifier("Wow, you really did that.")
# Sarcastic or impressed?

[{'label': 'POSITIVE', 'score': 0.9997573494911194}]

In [ ]:
# ➤ zero-shot-classification → no fine-tuning needed
# ➤ model = multi-label classifier (e.g., bart-large-mnli)
# ➤ input = unlabeled text
# ➤ candidate_labels = custom label set
# ➤ output → ranked labels by relevance + scores

from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "We're experimenting with reinforcement learning algorithms to optimize robotic movement.",
    candidate_labels=["machine learning", "robotics", "healthcare"],
)


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'sequence': "We're experimenting with reinforcement learning algorithms to optimize robotic movement.",
 'labels': ['machine learning', 'robotics', 'healthcare'],
 'scores': [0.6172025799751282, 0.37971267104148865, 0.0030847135931253433]}

In [ ]:
# ➤ text-generation → generates continuation for a given prompt
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to manage and track the results of your tests on an automated workflow, how to control and control all the tasks you perform within your tests workflow, how to provide and enable all the available options to improve your'}]

In [ ]:
from transformers import pipeline

# ➤ hyperparameters:
#     ↳ max_length = 30 → total number of tokens (prompt + generated)
#     ↳ num_return_sequences = 2 → generate 2 different completions
# ➤ other optional params (not used here):
#     ↳ temperature → controls randomness (↑ temp = more creative)
#     ↳ top_k / top_p → controls sampling diversity

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "In this course, we will teach you how to do all sorts of things based on the experiences you experience that you've had with the students. All"},
 {'generated_text': 'In this course, we will teach you how to create a model for model management, performance, and data analysis.\n\n\n\nWith your project'}]

In [ ]:
# ➤ fill-mask → predicts masked word in a sentence
# ➤ uses masked language models (e.g., BERT)
# ➤ <mask> = special token the model will fill
# ➤ top_k = number of predictions returned (ranked by confidence)

from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
unmasker("I am currently applying for a [MASK] internship.", top_k=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.19370311498641968,
  'token': 2621,
  'token_str': 'summer',
  'sequence': 'i am currently applying for a summer internship.'},
 {'score': 0.10473033785820007,
  'token': 11643,
  'token_str': 'modeling',
  'sequence': 'i am currently applying for a modeling internship.'}]

In [ ]:
# ➤ ner = Named Entity Recognition → detects entities in text
# ➤ grouped_entities=True → combines subword tokens into full entities
# ➤ output = list of detected entities with:
#     ↳ entity_group (e.g., PER, ORG, LOC)
#     ↳ word (actual entity text)
#     ↳ score (confidence), start/end (char positions)

from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("Barack Obama studied at Harvard University, worked in Washington, and now lives in Chicago with Michelle Obama.")


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'entity_group': 'PER',
  'score': np.float32(0.9991107),
  'word': 'Barack Obama',
  'start': 0,
  'end': 12},
 {'entity_group': 'ORG',
  'score': np.float32(0.951369),
  'word': 'Harvard University',
  'start': 24,
  'end': 42},
 {'entity_group': 'LOC',
  'score': np.float32(0.99944824),
  'word': 'Washington',
  'start': 54,
  'end': 64},
 {'entity_group': 'LOC',
  'score': np.float32(0.999298),
  'word': 'Chicago',
  'start': 83,
  'end': 90},
 {'entity_group': 'PER',
  'score': np.float32(0.99817735),
  'word': 'Michelle Obama',
  'start': 96,
  'end': 110}]

In [ ]:
# ➤ question-answering → extract answer from given context
# ➤ input = question + context paragraph
# ➤ model scans context and returns:
#     ↳ answer (text span)
#     ↳ score (confidence), start/end (char positions)

from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Which university did Alice graduate from?",
    context="Alice completed her studies in computer science at Stanford University in 2022.",
)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'score': 0.5476713180541992,
 'start': 51,
 'end': 70,
 'answer': 'Stanford University'}

In [ ]:
# summarizes given text
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    Raskolnikov had a terrible dream. He was a little boy, walking with his father through a desolate village.
    The sky was low and grey, and everything was silent. Suddenly, a drunken group of peasants appeared, laughing
    and shouting. They had a cart with an old, broken-down mare hitched to it. Laughing cruelly, they tried to
    force the exhausted animal to run. They whipped it, beat it with crowbars, screamed at it as it staggered and fell.
    The boy sobbed and begged them to stop, but they only laughed louder. The horse was beaten to death, its eyes
    wide with terror and blood on its flanks, while the little boy screamed and cried, powerless. Then he woke up,
    drenched in sweat, heart pounding, the scene burned into his mind.

    This dream haunted him. It was more than just a nightmare; it was a symbol, something deeper. He paced the floor,
    wrestling with the images and the weight of what he had done. Murder was not the clean, rational act he had imagined.
    It was chaos. It left behind blood, terror, and voices in the night. The idea that he was above morality,
    that he could commit a crime for a higher purpose — it now felt hollow. And yet, he could not let go of it.
    He clung to the belief that his theory had merit, even as guilt gnawed at the edges of his soul. His intellect
    and his conscience were at war, and he was the battlefield.
    """
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'summary_text': ' Raskolnikov had a terrible dream about beating a broken-down mare to death . He woke up, drenched in sweat, heart pounding, the scene burned into his mind . The idea that he was above morality, that he could commit a crime for a higher purpose — it now felt hollow .'}]

In [ ]:
# translate
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

Device set to use cpu


[{'translation_text': 'This course is produced by Hugging Face.'}]